# Ataques ao RSA

O criptosistema RSA baseia-se na dificuldade em determinar $p$ e $q$ dado um inteiro $N$, tal que $N=p*q$ com $p$ e $q$ primos.

In [128]:
class RSA:
    def __init__(self, l):
        self.l = l
        self.r = random_prime(2^(floor(l/2)))
        self.p = next_prime(2*self.r+1)
        q = self.p*self.r
        m = (self.p-1)*(self.r-1)
        k = randint(2,m)
        while gcd(m,k)!=1:
            k = randint(2,m)
        s = power_mod(k,-1,m)
        self.pubkey = (q,k)
        self.privkey = s

    def cifra(self,a):
        q,k = self.pubkey
        c = power_mod(a,k,q)
        return c

    def decifra(self, c):
        s = self.privkey
        q,_=self.pubkey
        z = power_mod(c,s,q)
        return z

    def assina(self,m):
        sig = power_mod(m,self.privkey,self.pubkey[0])
        return sig

    def verifica(self,m, sig):
        m_k = power_mod(sig,self.pubkey[1],self.pubkey[0])
        return (m_k == m)
    
class CopperSmith:
    def __init__(self, rsa):
        if(isinstance(rsa,RSA)):
            self.rsa = rsa
        else:
            raise('rsa parameter must be an object of class RSA')
    
    def reverse_pub_key(self, beta):
        N = Integers(self.rsa.pubkey[0])
        hbits = self.rsa.l//4 - 2
        hidden_bits = ZZ.random_element(2^(hbits-1), 2^hbits-1)
        leaked_bits = self.rsa.p - hidden_bits
        
        R.<x> = PolynomialRing(N, implementation='NTL')
        f = x + leaked_bits
        
        X = floor(self.rsa.pubkey[0] ^ (beta**2)) #f is of degree one, so d=1
        
        rts = f.small_roots(beta=beta, X=X)
        
        if len(rts)>0:
            recovered_hidden_b = rts[0]
            recovered_p = lift(leaked_bits+recovered_hidden_b)
            print(recovered_p == self.rsa.p)
            print(self.rsa.pubkey[0] // recovered_p == self.rsa.r)
    
    
    def reverse_pub_key_lattice(self, beta):
        N = Integers(self.rsa.pubkey[0])
        hbits = self.rsa.l//4 - 2
        hidden_bits = ZZ.random_element(2^(hbits-1), 2^hbits-1)
        leaked_bits = self.rsa.p - hidden_bits
        
        R.<x> = PolynomialRing(N, implementation='NTL')
        f = x + leaked_bits
        X = floor(self.rsa.pubkey[0] ^ (beta**2)) #f is of degree one, so d=1
        
        roots = map(QQ, [leaked_bits**i for i in range(2)])
        LW = matrix(QQ, 2, 1, roots)
        G=identity_matrix(QQ, 2).augment(LW)
        print(G)
        print(G.LLL())
        print(hidden_bits)
        
        rts = f.small_roots(beta=beta, X=X)
        if len(rts)>0:
            recovered_hidden_b = rts[0]
            recovered_p = lift(leaked_bits+recovered_hidden_b)
            print(recovered_p == self.rsa.p)
            print(self.rsa.pubkey[0] // recovered_p == self.rsa.r)
        

In [129]:
rsa = RSA(128)
copper = CopperSmith(rsa)
copper.reverse_pub_key_lattice(1/2)

[                  1                   0                   1]
[                  0                   1 2332308929521884002]
[                   1                    0                    1]
[-1166154464760942001                    1  1166154464760942001]
611893707
True
True


## Inversão da chave pública

## Inversão do criptograma

In [103]:
N = Integers(87)
N.order()

87